# Central Motivation for Project

The central goal of this project is to help students decide on three nutricious and "healthy" meals that are optimized for their needed nutritional intake. 

# Visualization of the Data

In [1]:
import requests
import pandas as pd

In [2]:
def find_periods(date):
    '''This function takes a date as input and returns the periods associated 
    with the date from the Dine On Campus API. We need to do this because the
    API changes the period id frequently. Periods represent the time of day
    (breakfast, lunch, dinner, etc.).
    
    Args:
        date (str): The date to search for in the format YYYY-M-DD
        
    Returns:
        periods_dict (dict): The dict of periods associated with the date; 
        empty if no periods found for date
    '''

    # create an empty dictionary to store the periods
    periods_dict = dict()

    # call the API to get the json information.
    url = "https://api.dineoncampus.com/v1/location/586d05e4ee596f6e6c04b527/periods?platform=0&date=" + date
    req = requests.get(url)

    # format the response as json
    data = req.json()

    # get the periods from the json
    periods = data.get("periods")

    # for each of the periods found
    for period in periods:
        # add the name and id to the dictionary
        periods_dict[period.get("name")] = period.get("id")

    # return the final dictionary
    return periods_dict


In [3]:
def get_menu(date, meal):
    '''Returns a dataframe containing the menu for a given date
    and all of its nutritional information.
    
    Args:
        date (str): The date to search for in the format YYYY-M-DD

    Returns:
        DataFrame: A dataframe containing the menu for the given date;
        empty if no menu found for date provided
    '''
    periods = find_periods(date)

    final_df = pd.DataFrame(columns=['food'])
    final_df = final_df.set_index('food')

    for period in periods.keys():
        if (period != meal):
            continue
        
        url = "https://api.dineoncampus.com/v1/location/586d05e4ee596f6e6c04b527/periods/" + periods[period] + "?platform=0&date=" + date
        req = requests.get(url)
        # get req as json
        data = req.json()
        # get the catagories of food
        catagories = data.get('menu').get('periods').get('categories')

        pd_food = pd.DataFrame(columns=['food'])
        pd_food = pd_food.set_index('food')

        # for each catagory
        for catagory in catagories:
            items = catagory.get('items')
            # for each food in the catagory
            for food in items:
                name = food.get('name')
                nutrients = food.get('nutrients')
                portion = food.get('portion')
                for nutrient in nutrients:
                    nutrient_name = nutrient.get('name')
                    nutrient_value = nutrient.get('value_numeric')
                    pd_food.at[name, nutrient_name] = nutrient_value
                pd_food.at[name, 'Serving Size'] = portion
        
        final_df = pd.concat([final_df, pd_food], sort=True)

    return final_df
    

In [4]:
# this takes a while to run due to the slow API from Dine On Campus
food_data_breakfast = get_menu("2023-3-24", "Breakfast")
food_data_lunch = get_menu("2023-3-24", "Lunch")
food_data_dinner = get_menu("2023-3-24", "Dinner")

In [5]:
import plotly.express as px

In [6]:
calories_breakfast = food_data_breakfast['Calories'].apply(pd.to_numeric)
fat_lunch = food_data_lunch['Total Fat (g)'].apply(pd.to_numeric)

In [7]:
fig = px.bar(calories_breakfast, x=calories_breakfast.index, y="Calories", title="Calories in Breakfast Foods", )
fig.show()

In [8]:
fig = px.bar(fat_lunch, x=fat_lunch.index, y="Total Fat (g)", title="Fat in Lunch Foods")
fig.show()

# What ML Tools Will be Used

One ML tool we could use is K-Means clustering. This could be used to group foods into catagories of "healthy" and "unhealthy" foods. This could bring up an issue that the model inacurately groups certain foods that are definetly "unhealthy" into the "healthy" ones. One fallback for this is to incude weighted instead of normalized features.

Another tool we could use is a Random Forest Classifier. We could use data from foods deemed "healthy" by the USDA Food and Nutrition services in order to classify foods into "healthy" and "unhealthy" foods. This can also be acheived by applying a "healthy" score and using a Random Forest Regressor. This relies on other data from another source, which could be a bit tedious to combine with our dataset.